<a href="https://colab.research.google.com/github/gustavolenin/Home_Credit_Default_Risk/blob/main/data_understanding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.0 Importações

In [ ]:
# !pip install --upgrade category_encoders

In [ ]:
import math
import datetime 
import numpy             as np
import matplotlib.pyplot as plt
import matplotlib        as mpl
import pandas            as pd
import seaborn           as sns
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score
from category_encoders import TargetEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from IPython.core.display import HTML
from IPython.display import Image
import warnings
warnings.filterwarnings("ignore")

# 0.1. Funções Auxiliares

In [ ]:
def load_csv(path):
    ''' Função utilizada para carregamento dos dados'''
    df = pd.read_csv(path, low_memory=False)
    return df

def show_dimensions(df):
    ''' Função utilizada para obter as dimensões dos dados'''
    print(f"Number of Rows:    {df.shape[0]}")
    print(f"Number of Columns: {df.shape[1]}")
    print(f"Shape:             {df.shape}")
    return None

def show_data_types(df):
    ''' Função utilizada para obter os tipos dos dados'''
    print(df.dtypes)
    return None

def check_na(df):
    ''' Função utilizada para visualizar dados ausentes ou NaN'''
    print(df.isna().sum())
    return None

def remove_space_column_name(df):
    ''' Função utilizada para remover espaços desnecessários nos nomes das colunas'''
    # Remover espaços desnecessários
    old_columns = df.columns
    remove_space = lambda x: x.strip()
    new_cols = list(map(remove_space, old_columns))
    # Renomear as colunas
    df.columns = new_cols
    return df

def remove_dollar_sign(df):
    ''' Função utlizada para remover símbolo de cifrão da coluna Income'''
    df.Income = df.Income.str.replace('$','')
    return df

def remove_end_space(df):
    ''' Função utilizada para remover espaço no final das strings presentes na coluna Income'''
    df.Income = df.Income.str.replace(' ','')
    return df

def remove_comma(df):
    ''' Função utilizada para remover vírgula nos dados da coluna Income'''
    df.Income = df.Income.str.replace(',','')
    return df
    
def rename_data_country(df):
    ''' Função utilizada para renomear os dados da coluna Country'''
    country_map = {'SP' : 'Spain', 'CA' : 'Canada', 'US' : 'United States', 'AUS' : 'Australia', 'GER' : 'Germany', 'IND' : 'India', 'SA' : 'Saudi Arabia', 'ME' : 'Montenegro'}
    df['Country'] = df['Country'].map(country_map)
    return df

def clean_data(df):
    ''' Função utilizada para realizar a limpeza nos dados'''
    # Primeira etapa da limpeza 
    df_step1 = remove_space_column_name(df)

    # Segunda etapa da limpeza
    df_step2 = remove_dollar_sign(df)
    
    # Terceira etapa da limpeza
    df_step3 = remove_end_space(df)

    # Quarta etapa da limpeza 
    df_step4 = remove_comma(df)

    # Quinta etapa da limpeza
    df_step5 = rename_data_country(df)

    return df_step5

def show_descriptive_statistical(df):
    ''' Função utilizada para exibir informações de estatística descritiva'''
    # Central Tendency - mean, median
    ct1 = pd.DataFrame(df.apply(np.mean)).T
    ct2 = pd.DataFrame(df.apply(np.median)).T

    # Dispersion - std, min, max, range, skew, kurtosis
    d1 = pd.DataFrame(df.apply(np.std)).T
    d2 = pd.DataFrame(df.apply(min)).T
    d3 = pd.DataFrame(df.apply(max)).T
    d4 = pd.DataFrame(df.apply(lambda x: x.max() - x.min())).T
    d5 = pd.DataFrame(df.apply(lambda x: x.skew())).T
    d6 = pd.DataFrame(df.apply(lambda x: x.kurtosis())).T

    df_descriptive = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
    df_descriptive.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
    return df_descriptive

def draw_scatterplot(list_feature,df,y):
    ''' Função utilizada para gerar scatterplots'''
    suptitle = 'Gráficos de Dispersão - '+str(y)
    fig, ax = plt.subplots(int(len(list_feature)/2),2, figsize=(20,40))
    colunas = list_feature
    indice = 0
    for j in range(len(ax)):
        for i in range(len(ax[j])):
            ax[j][i].tick_params(labelsize=12)
            ax[j][i] = sns.scatterplot(data=df, x=colunas[indice],y=y,ax=ax[j][i])
            indice += 1
    fig.suptitle(suptitle, position=(.5,1), fontsize=22)
    fig.tight_layout(pad=5)
    return
    
def draw_histplot(num_variables):
    ''' Função utilizada para gerar histogramas'''
    fig, ax = plt.subplots(int(len(num_variables.columns)/2),2, figsize=(15,15))
    colunas = num_variables.columns
    indice = 0
    for j in range(len(ax)):
        for i in range(len(ax[j])):
            ax[j][i].tick_params(labelsize=12)
            ax[j][i] = sns.histplot(data=num_variables[colunas[indice]],ax=ax[j][i])
            indice += 1
    fig.suptitle('Histogramas', position=(.5,1), fontsize=22)
    fig.tight_layout(pad=5)
    return

def plot_kde(num_attributes,df):
    ''' Função utilizada para gerar kdeplots'''
    fig, ax = plt.subplots(int(len(num_attributes)/2),2, figsize=(18,30))
    colunas = num_attributes
    indice = 0
    for j in range(len(ax)):
        for i in range(len(ax[j])):
            ax[j][i].tick_params(labelsize=12)
            ax[j][i] = sns.kdeplot(x=df[colunas[indice]], ax=ax[j][i],shade=True,hue=df.Response)
            indice += 1
    fig.suptitle('KDE Plots', position=(.5,1), fontsize=20)
    fig.tight_layout(pad=5)
    return

def plot_boxplot(num_attributes,df):
    ''' Função utilizada para gerar boxplots'''
    fig, ax = plt.subplots(int(len(num_attributes)/2),2, figsize=(20,50))
    colunas = num_attributes
    indice = 0
    for j in range(len(ax)):
        for i in range(len(ax[j])):
            ax[j][i].tick_params(labelsize=12)
            ax[j][i].tick_params(labelsize=10)
            ax[j][i] = sns.boxplot(data=df, ax=ax[j][i], x=df['Response'], y=df[colunas[indice]])
            indice += 1
    fig.suptitle('Boxplots', position=(.5,1), fontsize=20)
    fig.tight_layout(pad=5)
    return

def categorical_distribution_plot(df,category):
    ''' Função utilizada para gerar gráficos de distribuição de dados para variáveis categóricas'''
    fig, _ = plt.subplots(1,2, figsize=(20,5))
    fig.suptitle('Distribuição de Dados', position=(.50,1), fontsize=15)
    fig.tight_layout(pad=3)
    plt.subplot(1, 2, 1)
    sns.histplot(df[category])
    plt.subplot(1, 2, 2)
    sns.barplot(data=df,x=df.groupby([category]).Response.mean().sort_values(ascending=False).index,y=df.groupby([category]).Response.mean().sort_values(ascending=False).values)
    plt.ylabel('Percentual')
    plt.show()
    return

def create_weekday_column(df):
    ''' Função utilizada para criar a coluna Weekday, referente ao dia da semana em que o cliente se vinculou à empresa'''
    weekday_map = {0: 'Segunda-Feira', 1: 'Terça-Feira', 2: 'Quarta-Feira', 3: 'Quinta-Feira', 4: 'Sexta-Feira', 5: 'Sábado', 6: 'Domingo'}
    df['Weekday'] = df['Dt_Customer'].dt.weekday.map(weekday_map)
    return df

def create_month_column(df):
    ''' Função utilizada para criar a coluna Month, referente ao mês em que o cliente se vinculou à empresa'''
    month_map = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
    df['Month'] = df['Dt_Customer'].dt.month.map(month_map)
    return df

def create_year_column(df):
    ''' Função utilizada para criar a coluna Year, referente ao ano em que o cliente se vinculou à empresa'''
    df['Year'] = df['Dt_Customer'].dt.year
    return df

def create_age_column(df):
    ''' Função utilizada para criar a coluna Age, referente à idade do cliente'''
    ano_atual = datetime.datetime.today().year
    df['Age'] = ano_atual - df['Year_Birth']
    return df

def create_continent_column(df):
    ''' Função utilizada para criar a coluna Continent, referente ao continente do cliente'''
    continent_map = {'Spain': 'Europe', 'Canada': 'North-America', 'United States': 'North-America', 'Australia': 'Oceania', 'Germany': 'Europe', 'India': 'Saudi Arabia', 'Montenegro': 'Europe'}
    df['Continent'] = df['Country'].map(continent_map)
    return df

def jupyter_settings():
    ''' Função utilizada para configurar o notebook'''
    %matplotlib inline
    #%pylab inline
    
    #plt.style.use( 'ggplot')
    plt.rcParams['figure.figsize'] = [12, 8]
    plt.rcParams['font.size'] = 24
    mpl.rcParams['pdf.fonttype'] = 42
    mpl.rcParams['ps.fonttype'] = 42
    mpl.rcParams['font.family'] = 'Arial'
    
    display( HTML( '') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()
    return

In [ ]:
# Configurando o notebook
jupyter_settings()